In [10]:
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from tqdm.auto import tqdm

coords_path = 'data/2022-12-17 15:52:24.006277-coords.json'
data_path = 'data/2022-12-17 14:46:49.632892-output.json'
routes_path = 'data/2022-12-18 01:33:21.284286-company-routes.json'


mapbox_access_token = open("config/.mapbox_token").read()
px.set_mapbox_access_token(mapbox_access_token)

In [11]:
with open(coords_path, 'r') as fid:
    coords_data = json.load(fid)

with open(data_path, 'r') as fid:
    route_data = json.load(fid)

with open(routes_path, 'r') as fid:
    company_routes_data = json.load(fid)

In [13]:
records = []
for company, routes in company_routes_data['routes'].items():
    for route in routes:
        record = {'company': company, 'src': route[0], 'dst': route[1]}
        records.append(record)

df = pd.DataFrame(records)

available_slugs = set(coords_data['coords'].keys())
def normalize(s):
    for slug in available_slugs:
        if slug in s:
            return slug
        else:
            continue
    return None

df['src'] = df.src.map(normalize)
df['dst'] = df.dst.map(normalize)

df['src_x'] = df.src.map(lambda x: coords[x][0] if x in coords else None)
df['src_y'] = df.src.map(lambda x: coords[x][1] if x in coords else None)
df['dst_x'] = df.dst.map(lambda x: coords[x][0] if x in coords else None)
df['dst_y'] = df.dst.map(lambda x: coords[x][1] if x in coords else None)
df.head(5)


,company,src,dst,src_x,src_y,dst_x,dst_y
0,1001,sao-paulo-sp,rio-de-janeiro-rj,-46.633382,-23.550651,-43.209373,-22.911014
1,1001,sao-paulo-sp,cabo-frio-rj,-46.633382,-23.550651,-42.018923,-22.880437
2,1001,sao-paulo-sp,rio-de-janeiro-rj,-46.633382,-23.550651,-43.209373,-22.911014
3,1001,sao-paulo-sp,florianopolis-sc,-46.633382,-23.550651,-48.549610,-27.597300
4,1001,sao-paulo-sp,niteroi-rj,-46.633382,-23.550651,-43.114700,-22.888400


In [19]:

def slug_to_city(slug):
    splitted_slug = slug.split("-")
    uf = splitted_slug[-1].lower()
    place = " ".join(splitted_slug[0:-1])
    address = f'{place} / {uf}'
    return address

def generate_slug_df(df, slug):
    QUERY = f'(src == "{slug}" or dst == "{slug}")'
    fig_df = df.query(QUERY).copy(True)

    inds = (fig_df.dst == slug)
    orig_df = fig_df.copy(True)
    fig_df.loc[inds, 'src'] = orig_df[inds]['dst']
    fig_df.loc[inds, 'dst'] = orig_df[inds]['src']
    fig_df.loc[inds, 'src_x'] = orig_df[inds]['dst_x']
    fig_df.loc[inds, 'src_y'] = orig_df[inds]['dst_y']
    fig_df.loc[inds, 'dst_x'] = orig_df[inds]['src_x']
    fig_df.loc[inds, 'dst_y'] = orig_df[inds]['src_y']
    del orig_df
    return fig_df

distinct_slugs = set(df.src) | set(df.dst)

slug_dfs = {}
for slug in tqdm(distinct_slugs):
    slug_dfs[slug] = generate_slug_df(df, slug)

slug_df = pd.concat(slug_dfs.values())

100%|██████████| 2204/2204 [00:10<00:00, 205.53it/s]


In [22]:
fig_df = slug_df.query("src == 'ariquemes-ro'")

fig = px.scatter_mapbox(fig_df,
                     lon='dst_x',
                      lat='dst_y', 
                     color='company',
                     hover_name='dst',
                     height=800)

fig.update_traces(marker=dict(size=5))

fig.add_trace(go.Scattermapbox(
    name='source',
    lon = [fig_df.src_x.unique()[0]],
    lat = [fig_df.src_y.unique()[0]],
    mode = 'markers',
    marker = dict(
        symbol='square',
        size = 8,
        color = 'white'
    )
    )
    )

# fig.update_geos(fitbounds="locations", projection_type='orthographic',
#     resolution=50,
#     showcoastlines=True, coastlinecolor="RebeccaPurple",
#     showocean=True, oceancolor="LightBlue",
#     showlakes=True, lakecolor="LightBlue",
#     showrivers=True, rivercolor="LightBlue",
#         showcountries=True, countrycolor="Black")
fig.update_layout(
    mapbox = {
        'accesstoken': mapbox_access_token,
        'style': "dark", 'zoom': 3},
    showlegend = True)

fig.show()